In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

In [9]:
# ======================
# Config
# ======================
DATA_DIR = "cnn_dataset"
IMG_SIZE = 224
BATCH_SIZE = 32
# EPOCHS = 25
EPOCHS = 20
# LR = 1e-3
LR = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

NUM_CLASSES = 3
CLASS_NAMES = ["1509", "IRRI-6", "Super_White"]

# SAVE_PATH = "mobilenetv2_best.pth"
SAVE_PATH = "mobilenet_v2_finetuned_best.pth"

In [10]:
# ======================
# Transforms
# ======================
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

In [11]:
# ======================
# Datasets
# ======================
train_ds = datasets.ImageFolder(f"{DATA_DIR}/train", train_tfms)
val_ds   = datasets.ImageFolder(f"{DATA_DIR}/val", val_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print("Classes:", train_ds.classes)

Classes: ['1509', 'IRRI-6', 'Super White']


In [12]:
# ======================
# Model (MobileNetV2)
# ======================
model = models.mobilenet_v2(weights="DEFAULT")

# freeze backbone (optional but recommended first)
for param in model.features.parameters():
    param.requires_grad = True

# replace classifier
model.classifier[1] = nn.Linear(model.last_channel, NUM_CLASSES)

model = model.to(DEVICE)

In [13]:
# ======================
# Loss & optimizer
# ======================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [14]:
# ======================
# Training loop
# ======================
best_acc = 0

for epoch in range(EPOCHS):

    # -------- TRAIN --------
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):

        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(imgs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total

    # -------- VALIDATION --------
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in val_loader:

            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)

            _, preds = torch.max(outputs, 1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total

    print(f"\nTrain Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # save best
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), SAVE_PATH)
        print("✅ Best model saved")

print("\n🔥 Training finished")

Epoch 1/20: 100%|██████████| 282/282 [02:53<00:00,  1.62it/s]



Train Acc: 0.8563 | Val Acc: 0.9389
✅ Best model saved


Epoch 2/20: 100%|██████████| 282/282 [02:43<00:00,  1.73it/s]



Train Acc: 0.9471 | Val Acc: 0.9544
✅ Best model saved


Epoch 3/20: 100%|██████████| 282/282 [01:39<00:00,  2.84it/s]


...


Train Acc: 0.9886 | Val Acc: 0.9744


Epoch 19/20: 100%|██████████| 282/282 [00:59<00:00,  4.71it/s]



Train Acc: 0.9908 | Val Acc: 0.9778


Epoch 20/20: 100%|██████████| 282/282 [00:59<00:00,  4.72it/s]



Train Acc: 0.9887 | Val Acc: 0.9711

🔥 Training finished
